In [ ]:
import pandas as pd
import re
from geopy.geocoders import Nominatim #used for geolocating cities and attractions for route planning

In [3]:
attractions = pd.read_csv('C:/Users/turne/OneDrive/Desktop/TripleTen Project Files/cleaned_data_USA.csv')
roadside = pd.read_csv('C:/Users/turne/OneDrive/Desktop/TripleTen Project Files/us_roadside_attractions.csv')

In [ ]:
attractions = attractions.drop(columns = ['Unnamed: 0', 'categories', 'broader_category', 'zipcode', 'All_Cities'])
attractions = attractions.rename(columns ={'Weighted_Score': 'weighted_score', 'Weighted_Average': 'weighted_average'})

KeyError: "['Unnamed: 0', 'categories', 'broader_category', 'zipcode', 'All_Cities'] not found in axis"

In [5]:
print(attractions.duplicated().sum(), attractions.isnull().sum())

0 name                0
main_category       0
rating              0
reviews             0
address             0
city                0
country             0
state               0
weighted_score      0
weighted_average    0
dtype: int64


No duplicates and no missing data in this dataset, now we'll check the next one

In [6]:
roadside.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11710 entries, 0 to 11709
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   index       11710 non-null  int64 
 1   title       11710 non-null  object
 2   year        11710 non-null  object
 3   image       11707 non-null  object
 4   image_link  11710 non-null  object
dtypes: int64(1), object(4)
memory usage: 457.6+ KB


In [7]:
roadside = roadside.drop(columns = ['index', 'image'])
roadside.drop_duplicates(inplace=True)
roadside = roadside.rename(columns = {'title': 'name', 'year': 'year_photo_taken'})
roadside['year_photo_taken'] = pd.to_numeric(roadside['year_photo_taken'], errors='coerce')
roadside['year_photo_taken'] = roadside['year_photo_taken'].astype('Int64')
roadside.info()

<class 'pandas.core.frame.DataFrame'>
Index: 11708 entries, 0 to 11709
Data columns (total 3 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   name              11708 non-null  object
 1   year_photo_taken  11685 non-null  Int64 
 2   image_link        11708 non-null  object
dtypes: Int64(1), object(2)
memory usage: 377.3+ KB


Handled duplicate data, dropped redundant index column and image column, and converted 'year_photo_taken' to int64

In [8]:
#create regex function to extract state from name of roadside attraction, used to merge both df's together
def state_name(name):
    match = re.search(r',\s*([A-Za-z ]+)$', name)
    if match:
        return match.group(1).strip()
    return None

roadside['state'] = roadside['name'].apply(state_name)
roadside.head()

,name,year_photo_taken,image_link,state
0,"Teapot Dome gas station, Zillah, Washington",1987,https://cdn.loc.gov/service/pnp/mrg/00000/0000...,Washington
1,"Harold's Auto Center, horizontal view, Sinclai...",1979,https://cdn.loc.gov/service/pnp/mrg/00000/0000...,Florida
2,Premium gas station (formerly Joy gas station)...,1977,https://cdn.loc.gov/service/pnp/mrg/00000/0000...,Canada
3,"Bomber gas station, diagonal view, Route 99 E....",1980,https://cdn.loc.gov/service/pnp/mrg/00000/0000...,Oregon
4,"Hat n' Boots gas station, overall view, Route ...",1980,https://cdn.loc.gov/service/pnp/mrg/00000/0000...,None


In [9]:
#create regex function to extract city from name of roadside attraction
def city_name(name):
    match = re.search(r',\s*([^,]+), \s*[A-Za-z ]+$', name)
    if match:
        return match.group(1).strip()
    return None

roadside['city'] = roadside['name'].apply(city_name)
roadside.head()

,name,year_photo_taken,image_link,state,city
0,"Teapot Dome gas station, Zillah, Washington",1987,https://cdn.loc.gov/service/pnp/mrg/00000/0000...,Washington,Zillah
1,"Harold's Auto Center, horizontal view, Sinclai...",1979,https://cdn.loc.gov/service/pnp/mrg/00000/0000...,Florida,Spring Hill
2,Premium gas station (formerly Joy gas station)...,1977,https://cdn.loc.gov/service/pnp/mrg/00000/0000...,Canada,Ontario
3,"Bomber gas station, diagonal view, Route 99 E....",1980,https://cdn.loc.gov/service/pnp/mrg/00000/0000...,Oregon,Milwaukie
4,"Hat n' Boots gas station, overall view, Route ...",1980,https://cdn.loc.gov/service/pnp/mrg/00000/0000...,None,None


Here I created two regex functions to extract the state and city names from each attraction's name

In [10]:
roadside = roadside.dropna().reset_index(drop=True)
roadside.head()

,name,year_photo_taken,image_link,state,city
0,"Teapot Dome gas station, Zillah, Washington",1987,https://cdn.loc.gov/service/pnp/mrg/00000/0000...,Washington,Zillah
1,"Harold's Auto Center, horizontal view, Sinclai...",1979,https://cdn.loc.gov/service/pnp/mrg/00000/0000...,Florida,Spring Hill
2,Premium gas station (formerly Joy gas station)...,1977,https://cdn.loc.gov/service/pnp/mrg/00000/0000...,Canada,Ontario
3,"Bomber gas station, diagonal view, Route 99 E....",1980,https://cdn.loc.gov/service/pnp/mrg/00000/0000...,Oregon,Milwaukie
4,"Hat n' Boots gas station (1945), boot restroom...",1977,https://cdn.loc.gov/service/pnp/mrg/00000/0000...,Washington,Seattle


In [11]:
attractions.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3124 entries, 0 to 3123
Data columns (total 10 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   name              3124 non-null   object 
 1   main_category     3124 non-null   object 
 2   rating            3124 non-null   float64
 3   reviews           3124 non-null   float64
 4   address           3124 non-null   object 
 5   city              3124 non-null   object 
 6   country           3124 non-null   object 
 7   state             3124 non-null   object 
 8   weighted_score    3124 non-null   float64
 9   weighted_average  3124 non-null   float64
dtypes: float64(4), object(6)
memory usage: 244.2+ KB


In [12]:
merged_attractions = roadside.merge(attractions, on='state', how='left')
merged_attractions.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11621 entries, 0 to 11620
Data columns (total 14 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   name_x            11621 non-null  object 
 1   year_photo_taken  11621 non-null  Int64  
 2   image_link        11621 non-null  object 
 3   state             11621 non-null  object 
 4   city_x            11621 non-null  object 
 5   name_y            0 non-null      object 
 6   main_category     0 non-null      object 
 7   rating            0 non-null      float64
 8   reviews           0 non-null      float64
 9   address           0 non-null      object 
 10  city_y            0 non-null      object 
 11  country           0 non-null      object 
 12  weighted_score    0 non-null      float64
 13  weighted_average  0 non-null      float64
dtypes: Int64(1), float64(4), object(9)
memory usage: 1.3+ MB


Merged both df's to get them ready for the geolocator function

In [13]:
merged_attractions = merged_attractions.drop(columns = ['city_y', 'name_y'])
merged_attractions = merged_attractions.rename(columns = {'city_x': 'city', 'name_x': 'name'})

In [14]:
merged_attractions['state'].value_counts().sort_values(ascending=False).head(10)

state
New York          1841
New Jersey        1203
California         936
Florida            610
Texas              390
Michigan           388
South Carolina     378
Montana            284
Georgia            276
South Dakota       268
Name: count, dtype: int64

Sorted by the states with the most attractions from the dataset, then discussed with my team which state we would focus on for our app. We decided on California

In [16]:
ca_attractions = merged_attractions[merged_attractions['state'] == 'California']
ca_attractions.head()

,name,year_photo_taken,image_link,state,city,main_category,rating,reviews,address,country,weighted_score,weighted_average
5,World's Largest Redwood Tree Service Station (...,1991,https://cdn.loc.gov/service/pnp/mrg/00000/0000...,California,Ukiah,NaN,NaN,NaN,NaN,NaN,NaN,NaN
20,"Gordon Spencer gas station, 4691 Huntington Dr...",1977,https://cdn.loc.gov/service/pnp/mrg/00000/0002...,California,South Pasadena,NaN,NaN,NaN,NaN,NaN,NaN,NaN
35,"Mattoon Service Station (pre-fabricated), angl...",1977,https://cdn.loc.gov/service/pnp/mrg/00000/0003...,California,Culver City,NaN,NaN,NaN,NaN,NaN,NaN,NaN
36,"Richfield Gas, Route 66, Cucamonga, California",1978,https://cdn.loc.gov/service/pnp/mrg/00000/0003...,California,Cucamonga,NaN,NaN,NaN,NaN,NaN,NaN,NaN
50,"Golden Eagle gas pumps, 8th & Market Streets, ...",1977,https://cdn.loc.gov/service/pnp/mrg/00000/0005...,California,San Diego,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
geolocator = Nominatim(user_agent="roadtrip_geocoder")

def geocode_location(row):
    city = row['city']
    state = row['state']
    if pd.notnull(city) and pd.notnull(state):
        location_str = f"{city}, {state}"
        try:
            location = geolocator.geocode(location_str)
            if location:
                return pd.Series([location.latitude, location.longitude])
        except:
            pass
    return pd.Series([None, None])

#Applies geocoding to the entire ca_attractions dataframe (this may take a while)
ca_attractions[['latitude', 'longitude']] = ca_attractions.apply(geocode_location, axis=1)

#Removes rows where latitude or longitude is NaN
ca_attractions = ca_attractions.dropna(subset=['latitude', 'longitude'])

ca_attractions.head()

C:\Users\turne\AppData\Local\Temp\ipykernel_5560\1670354864.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ca_attractions[['latitude', 'longitude']] = ca_attractions.apply(geocode_location, axis=1)
C:\Users\turne\AppData\Local\Temp\ipykernel_5560\1670354864.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ca_attractions[['latitude', 'longitude']] = ca_attractions.apply(geocode_location, axis=1)


,name,year_photo_taken,image_link,state,city,main_category,rating,reviews,address,country,weighted_score,weighted_average,latitude,longitude
5,World's Largest Redwood Tree Service Station (...,1991,https://cdn.loc.gov/service/pnp/mrg/00000/0000...,California,Ukiah,NaN,NaN,NaN,NaN,NaN,NaN,NaN,39.150166,-123.207786
20,"Gordon Spencer gas station, 4691 Huntington Dr...",1977,https://cdn.loc.gov/service/pnp/mrg/00000/0002...,California,South Pasadena,NaN,NaN,NaN,NaN,NaN,NaN,NaN,34.113306,-118.147829
35,"Mattoon Service Station (pre-fabricated), angl...",1977,https://cdn.loc.gov/service/pnp/mrg/00000/0003...,California,Culver City,NaN,NaN,NaN,NaN,NaN,NaN,NaN,34.021122,-118.396466
36,"Richfield Gas, Route 66, Cucamonga, California",1978,https://cdn.loc.gov/service/pnp/mrg/00000/0003...,California,Cucamonga,NaN,NaN,NaN,NaN,NaN,NaN,NaN,34.099199,-117.602210
51,"Far-Go gas pumps, Main Street, Barstow, Califo...",1979,https://cdn.loc.gov/service/pnp/mrg/00000/0005...,California,Barstow,NaN,NaN,NaN,NaN,NaN,NaN,NaN,34.898621,-117.024431


In [22]:
ca_attractions.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 886 entries, 0 to 885
Data columns (total 13 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   name              886 non-null    object 
 1   image_link        886 non-null    object 
 2   state             886 non-null    object 
 3   city              886 non-null    object 
 4   main_category     0 non-null      object 
 5   rating            0 non-null      float64
 6   reviews           0 non-null      float64
 7   address           0 non-null      object 
 8   country           0 non-null      object 
 9   weighted_score    0 non-null      float64
 10  weighted_average  0 non-null      float64
 11  latitude          886 non-null    float64
 12  longitude         886 non-null    float64
dtypes: float64(6), object(7)
memory usage: 90.1+ KB


In [23]:
ca_attractions = ca_attractions.drop(columns = ['main_category', 'rating', 'reviews',
                                                'address', 'country', 'weighted_score',
                                                'weighted_average']).reset_index(drop=True)
ca_attractions.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 886 entries, 0 to 885
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   name        886 non-null    object 
 1   image_link  886 non-null    object 
 2   state       886 non-null    object 
 3   city        886 non-null    object 
 4   latitude    886 non-null    float64
 5   longitude   886 non-null    float64
dtypes: float64(2), object(4)
memory usage: 41.7+ KB


Dropped columns not necessary for our app's function

In [24]:
ca_attractions.head()

,name,image_link,state,city,latitude,longitude
0,World's Largest Redwood Tree Service Station (...,https://cdn.loc.gov/service/pnp/mrg/00000/0000...,California,Ukiah,39.150166,-123.207786
1,"Gordon Spencer gas station, 4691 Huntington Dr...",https://cdn.loc.gov/service/pnp/mrg/00000/0002...,California,South Pasadena,34.113306,-118.147829
2,"Mattoon Service Station (pre-fabricated), angl...",https://cdn.loc.gov/service/pnp/mrg/00000/0003...,California,Culver City,34.021122,-118.396466
3,"Richfield Gas, Route 66, Cucamonga, California",https://cdn.loc.gov/service/pnp/mrg/00000/0003...,California,Cucamonga,34.099199,-117.602210
4,"Far-Go gas pumps, Main Street, Barstow, Califo...",https://cdn.loc.gov/service/pnp/mrg/00000/0005...,California,Barstow,34.898621,-117.024431


In [25]:
ca_attractions.to_csv('california_service_stations_cleaned.csv', index = False)

Exported to csv for upload to Git repo